In [4]:
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import Union

In [12]:
def get_Van_b():
    """
    Gets the Vandermonde matrix and b we are considering.
    
    """
    n = 15
    m = 100

    alphas = np.array([i/(m - 1) for i in range(m)])
    
    Van = np.ndarray((m, n))
    for i in range(m):
        for j in range(n):
            Van[i, j] = alphas[i]**j
    b = np.zeros(m)
    for i in range(m):
        b[i] = math.exp(math.sin(4 * alphas[i]))
        
    return Van, b

Van, b = get_Van_b()

In [9]:
def back_substitution(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    n = b.size
    x = np.zeros_like(b)

    if A[n-1, n-1] == 0:
        raise ValueError


    x[n-1] = b[n-1]/A[n-1, n-1]
    C = np.zeros((n,n))
    for i in range(n-2, -1, -1):
        bb = 0
        for j in range (i+1, n):
            bb += A[i, j]*x[j]

        C[i, i] = b[i] - bb
        x[i] = C[i, i]/A[i, i]

    return x

In [80]:
def calculate_Hausholder(v: np.array):
    """
    Helper function to calculate the Hausholder matrix from v.
    
    Parameters
    ----------
    v : np.array
        The given vector. Needs to be normalized prior to passing.
        
    Returns
    -------
    H : np.ndarray
        The Hausholder matrix of v.
    """
    H = np.identity(max(v.shape)) - 2*np.outer(v, v)
    return H

def recover_Qis(A: np.ndarray):
    """
    Calculates all Q_is from a Hausholder applied matrix A.
    
    Parameters
    ----------
    
    A : np.ndarray
        The matrix on which Hausholder QR has been performed
        
    Returns
    -------
    Qis : list(np.ndarray)
        A list containing all Q's 
    """
    Qis = []
    
    m, n = A.shape
    
    for i in range(n):
        v = A[i+1:, i].copy()
        Q = np.identity(m-1)
        Q[i:, i:] = calculate_Hausholder(v)
        Qis.append(Q)
    
    return Qis

def calculate_Q(A: np.ndarray):
    """
    Returns the orthonormal matrix Q from the matrix A after Hausholder QR 
    is applied.
    
    Parameters
    ----------
    
    A : np.ndarray
        The matrix on which Hausholder QR has been performed

    Returns
    -------    
    
    Q: np.ndarray
        The orthonormal matrix involved in QR
    """
    
    Qis = recover_Qis(A.copy())
    Q = np.identity(Qis[0].shape[0])
    for Qi in Qis:
        Q = np.matmul(Q, Qi.T)
        
    return Q


def householder_vectorized(a):
    """Use this version of householder to reproduce the output of np.linalg.qr 
    exactly (specifically, to match the sign convention it uses)
    
    based on https://rosettacode.org/wiki/QR_decomposition#Python
    """
    v = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))
    v[0] = 1
    tau = 2 / (v.T @ v)
    
    return v,tau

def qr_decomposition(A: np.ndarray) -> Union[np.ndarray, np.ndarray]:
    m,n = A.shape
    R = A.copy()
    Q = np.identity(m)
    
    for j in range(0, n):
        # Apply Householder transformation.
        v, tau = householder_vectorized(R[j:, j, np.newaxis])
        
        H = np.identity(m)
        H[j:, j:] -= tau * (v @ v.T)
        R = H @ R
        Q = H @ Q
        
    return Q[:n].T, np.triu(R[:n])

In [90]:
n = 15
m = 100

#Calcualting QR
A = qr_decomposition_haus(Van)
Q = calculate_Q(A)
Q = Q[:, :n]
Q_C, R_C = np.linalg.qr(Van)

R = np.triu(A)

#Take nonzero rows of R. Since Van is full rank, will be n rows
R = np.array([x for x in R if x.any() != 0])

#Check if I did it right
assert R.shape == (n, n), "Oops"

abs(Q_C - Q) < 0.01

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [89]:
Q

array([[-1.        , -0.40737074, -0.55755301, ..., -0.16767204,
        -0.14889781, -0.0196807 ],
       [-0.18181818, -1.0370337 , -0.41555122, ...,  0.1904221 ,
         0.2193933 ,  0.43897979],
       [-0.18181818, -0.29555537, -1.09784934, ..., -0.43582459,
        -0.40509359, -0.20199975],
       ...,
       [-0.18181818,  0.27559715, -0.34228511, ...,  0.34169551,
        -0.57052107,  0.0503285 ],
       [-0.18181818,  0.28160929, -0.36460823, ..., -0.55847513,
         0.33559426, -0.44313711],
       [-0.18181818,  0.28762142, -0.38739918, ...,  0.10003157,
        -0.37042923, -0.6024131 ]])

In [91]:
c = np.matmul(Q[:, :n].T, b)

#Solution to part i 
x_soli = back_substitution(R, c)
print("Approximate solution to c_14: {} ".format(x_soli[-1]))

Approximate solution to c_14: -11722843.91526459 
